<a href="https://colab.research.google.com/github/jcyeh/dl_scripts/blob/master/fizzbuzz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FizzBuzz in TensorFlow (v2.0.0)

This Jupyter notebook demonstrates the FizzBuzz code in Python (v3.7.7)

In [1]:
import numpy as np
import tensorflow as tf
import time
import os

DEVICE = "GPU"

In [2]:
if DEVICE == "GPU":
    os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
    tf.config.set_soft_device_placement(True)
    tf.debugging.set_log_device_placement(enabled=True)

    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        # Restrict TensorFlow to only use the first GPU
        try:
            tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
        except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
            print(e)
elif DEVICE == "TPU":
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)
print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:CPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [3]:
# Represent each input by an array of its binary digits.
def binary_encode(i, num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])

In [4]:
# One-hot encode the desired outputs: [number, "fizz", "buzz", "fizzbuzz"]
def fizz_buzz_encode(i):
    if i % 15 == 0: return np.array([0, 0, 0, 1])
    elif i % 5 == 0: return np.array([0, 0, 1, 0])
    elif i % 3 == 0: return np.array([0, 1, 0, 0])
    else: return np.array([1, 0, 0, 0])

In [5]:
NUM_DIGITS = 10

x_train = np.array([binary_encode(i, NUM_DIGITS) for i in range(101, 2 ** NUM_DIGITS)])
y_train = np.array([fizz_buzz_encode(i) for i in range(101, 2 ** NUM_DIGITS)])

x_test = np.array([binary_encode(i, NUM_DIGITS) for i in range(1, 101)])
y_test = np.array([fizz_buzz_encode(i) for i in range(1, 101)])

In [6]:
if DEVICE == "GPU":
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1000, input_shape=(10,)))
    model.add(tf.keras.layers.Activation(tf.nn.relu))
    model.add(tf.keras.layers.Dense(4))
    model.add(tf.keras.layers.Activation(tf.nn.softmax))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
elif DEVICE == "TPU":
    with strategy.scope():
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(1000, input_shape=(10,)))
        model.add(tf.keras.layers.Activation(tf.nn.relu))
        model.add(tf.keras.layers.Dense(4))
        model.add(tf.keras.layers.Activation(tf.nn.softmax))
        model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarIsInitializedOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op LogicalNot in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Assert in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/tas

In [7]:
start = time.time()
model.fit(x_train, y_train, batch_size=64, nb_epoch=120, verbose=0)
end = time.time()
print("Traing Time: {0}s".format(end-start))

Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ModelDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AnonymousIteratorV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Execu

In [8]:
start = time.time()
result = model.evaluate(x_test, y_test, verbose=0)
end = time.time()
print("Acc: {0:.2f} ".format(result[1]))
print("Evaluation Time: {0}s".format(end-start))

Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op __inference_distributed_function_6501 in device /job:localhost/replica:0/task:0/device:CPU:0
Acc: 1.00
Evaluation Time: 0.20752286911010742s
